In [2]:
import cv2
import torch
import torch.nn.functional as F

from model_hourglass import StackedHourGlass
from utils import to_cuda, get_coords
from visualize import visualize
import numpy as np

#import plotly

#plotly.tools.set_credentials_file(username='', api_key='')

#import plotly.plotly as py
#import plotly.graph_objs as go

In [3]:
#path = 'results/test_images_128_bicubic/img3_64.jpg'
path = 'sample_images/0007_01.jpg'

img = cv2.imread(path, cv2.IMREAD_COLOR)
img = cv2.resize(img, (128, 128), interpolation=cv2.INTER_AREA)
img = torch.from_numpy(img.transpose(2, 0, 1)).float()
img = (img - 128) / 128.0
img.shape

torch.Size([3, 128, 128])

In [4]:
saved_model = 'trained_models/hourglass_55_epochs'

model = StackedHourGlass(nChannels=224, nStack=2, nModules=2, numReductions=4, nOutputs=200)
model.cuda()
model.load_state_dict(torch.load(saved_model))
model.eval()

img = to_cuda(img.unsqueeze(0), True)
out = F.sigmoid(model(img))

img = img.squeeze(0).cpu()
out = out.detach().squeeze(0).cpu()

In [5]:
# Visualization

thr, sz = 0.9, 0.25

x, y, z = get_coords(out.numpy(), sz=sz, thr=thr)

visualize(img, out, sz=0.25, thr=thr)

Image shape: (3, 128, 128)
Volume shape: (200, 128, 128)


In [17]:
# Visualization with Pyplot

#trace = go.Mesh3d(x=x,y=y,z=z,
#                   alphahull=-1,
#                   opacity=0.4,
#                   color='#00FFFF')
#py.iplot([trace])